In [95]:
!pip install catboost

     ---------------------------------------- 0.0/101.0 MB ? eta -:--:--
     ---------------------------------------- 0.1/101.0 MB 3.3 MB/s eta 0:00:31
     ---------------------------------------- 0.2/101.0 MB 2.5 MB/s eta 0:00:41
     ---------------------------------------- 0.3/101.0 MB 2.6 MB/s eta 0:00:39
     ---------------------------------------- 0.4/101.0 MB 2.3 MB/s eta 0:00:44
     ---------------------------------------- 0.5/101.0 MB 2.3 MB/s eta 0:00:43
     ---------------------------------------- 0.6/101.0 MB 2.2 MB/s eta 0:00:46
     ---------------------------------------- 0.7/101.0 MB 2.2 MB/s eta 0:00:45
     ---------------------------------------- 0.8/101.0 MB 2.3 MB/s eta 0:00:44
     ---------------------------------------- 0.9/101.0 MB 2.4 MB/s eta 0:00:42
     ---------------------------------------- 1.1/101.0 MB 2.4 MB/s eta 0:00:43
     ---------------------------------------- 1.2/101.0 MB 2.4 MB/s eta 0:00:43
      --------------------------------------- 1


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


     ----------------------- --------------- 62.1/101.0 MB 1.7 MB/s eta 0:00:23
     ------------------------ -------------- 62.2/101.0 MB 1.7 MB/s eta 0:00:23
     ------------------------ -------------- 62.3/101.0 MB 1.7 MB/s eta 0:00:23
     ------------------------ -------------- 62.4/101.0 MB 1.7 MB/s eta 0:00:23
     ------------------------ -------------- 62.5/101.0 MB 1.7 MB/s eta 0:00:23
     ------------------------ -------------- 62.6/101.0 MB 1.7 MB/s eta 0:00:23
     ------------------------ -------------- 62.8/101.0 MB 1.7 MB/s eta 0:00:23
     ------------------------ -------------- 62.9/101.0 MB 1.7 MB/s eta 0:00:23
     ------------------------ -------------- 63.0/101.0 MB 1.7 MB/s eta 0:00:23
     ------------------------ -------------- 63.1/101.0 MB 1.7 MB/s eta 0:00:23
     ------------------------ -------------- 63.2/101.0 MB 1.7 MB/s eta 0:00:22
     ------------------------ -------------- 63.3/101.0 MB 1.7 MB/s eta 0:00:22
     ------------------------ ----------

In [97]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor

In [107]:
data = pd.read_csv('HW_train_main_data.csv')
data_add = pd.read_csv('HW_train_additional_data.csv')
data_test = pd.read_csv('HW_test_main_data.csv')
data_test_add = pd.read_csv('HW_test_additional_data.csv')

In [108]:
data

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,apartment condition,sub_area,price
0,30392,2014-12-30,60,36.0,6.0,9.0,1.0,1974.0,3.0,5.0,3.0,9,9711800.0
1,36540,2013-06-28,44,29.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,19,6643200.0
2,94669,2013-09-15,62,60.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,80,5698300.0
3,17531,2013-12-13,33,18.0,4.0,9.0,1.0,1970.0,1.0,6.0,2.0,15,5055000.0
4,32159,2014-02-13,37,14.0,1.0,17.0,1.0,1991.0,1.0,1.0,2.0,74,5425000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28995,85589,2013-05-08,58,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,101,4330300.0
28996,11345,2014-04-12,38,NaN,9.0,17.0,1.0,NaN,1.0,1.0,NaN,134,4779900.0
28997,55519,2013-09-23,55,30.0,9.0,16.0,1.0,2002.0,2.0,8.0,3.0,38,7260000.0
28998,17196,2013-11-19,37,19.0,3.0,16.0,5.0,1979.0,1.0,9.0,3.0,43,6018000.0


In [109]:
data['build_year'] = data['build_year'].apply(lambda x : x if 1860 <= x <= 2018 else float('nan'))
data_test['build_year'] = data_test['build_year'].apply(lambda x : x if 1860 <= x <= 2018 else float('nan'))

In [110]:
data['timestamp'] = pd.to_datetime(data['timestamp'])
data_test['timestamp'] = pd.to_datetime(data_test['timestamp'])

In [111]:
data['year'] = data['timestamp'].dt.year
data['month'] = data['timestamp'].dt.month


In [112]:
data_test['year'] = data_test['timestamp'].dt.year
data_test['month'] = data_test['timestamp'].dt.month

In [113]:
merged_data = pd.merge(data, data_add)

In [114]:
merged_data.drop(columns=['timestamp'], inplace=True)


In [116]:
merged_data

,id,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,apartment condition,...,additional_education_facilities,culture_objects_facilities,shopping_centers_facilities,office_num,green_part,prom_part,cafe_count,church_facilities,mosque,leisure_facilities
0,30392,60,36.0,6.0,9.0,1.0,1974.0,3.0,5.0,3.0,...,3,0,16,1,2.86,0.00,0,0,0,0
1,36540,44,29.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,...,3,0,11,5,9.26,2.71,1,0,0,0
2,94669,62,60.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2,0,4,2,13.80,0.00,4,0,0,0
3,17531,33,18.0,4.0,9.0,1.0,1970.0,1.0,6.0,2.0,...,4,0,2,0,7.46,0.00,1,0,0,0
4,32159,37,14.0,1.0,17.0,1.0,1991.0,1.0,1.0,2.0,...,4,0,1,0,0.00,0.00,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28995,85589,58,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,3,0,1,0,9.26,7.02,0,0,0,0
28996,11345,38,NaN,9.0,17.0,1.0,NaN,1.0,1.0,NaN,...,16,10,23,141,12.24,0.00,71,15,0,0
28997,55519,55,30.0,9.0,16.0,1.0,2002.0,2.0,8.0,3.0,...,4,0,4,4,12.32,0.00,0,0,0,0
28998,17196,37,19.0,3.0,16.0,5.0,1979.0,1.0,9.0,3.0,...,3,0,11,6,39.80,7.01,3,0,0,0


In [80]:
y = merged_data['price']
X_a = merged_data.drop(columns=['price'])

In [117]:
imputer = IterativeImputer(random_state=42)
me = pd.DataFrame(imputer.fit_transform(merged_data), columns=merged_data.columns)

In [118]:
me

,id,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,apartment condition,...,additional_education_facilities,culture_objects_facilities,shopping_centers_facilities,office_num,green_part,prom_part,cafe_count,church_facilities,mosque,leisure_facilities
0,30392.0,60.0,36.000000,6.0,9.000000,1.000000,1974.000000,3.000000,5.000000,3.000000,...,3.0,0.0,16.0,1.0,2.86,0.00,0.0,0.0,0.0,0.0
1,36540.0,44.0,29.000000,9.0,13.865723,1.555441,1986.759037,1.663412,7.295711,2.464101,...,3.0,0.0,11.0,5.0,9.26,2.71,1.0,0.0,0.0,0.0
2,94669.0,62.0,60.000000,4.0,11.018363,1.635896,1985.160837,2.342162,6.786302,2.164586,...,2.0,0.0,4.0,2.0,13.80,0.00,4.0,0.0,0.0,0.0
3,17531.0,33.0,18.000000,4.0,9.000000,1.000000,1970.000000,1.000000,6.000000,2.000000,...,4.0,0.0,2.0,0.0,7.46,0.00,1.0,0.0,0.0,0.0
4,32159.0,37.0,14.000000,1.0,17.000000,1.000000,1991.000000,1.000000,1.000000,2.000000,...,4.0,0.0,1.0,0.0,0.00,0.00,3.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28995,85589.0,58.0,34.810948,2.0,9.763230,1.903057,1987.783226,1.918817,5.452792,1.852972,...,3.0,0.0,1.0,0.0,9.26,7.02,0.0,0.0,0.0,0.0
28996,11345.0,38.0,31.951249,9.0,17.000000,1.000000,2006.007224,1.000000,1.000000,1.365329,...,16.0,10.0,23.0,141.0,12.24,0.00,71.0,15.0,0.0,0.0
28997,55519.0,55.0,30.000000,9.0,16.000000,1.000000,2002.000000,2.000000,8.000000,3.000000,...,4.0,0.0,4.0,4.0,12.32,0.00,0.0,0.0,0.0,0.0
28998,17196.0,37.0,19.000000,3.0,16.000000,5.000000,1979.000000,1.000000,9.000000,3.000000,...,3.0,0.0,11.0,6.0,39.80,7.01,3.0,0.0,0.0,0.0


In [119]:
y = me['price']
X = me.drop(columns=['price'])

In [120]:
X


,id,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,apartment condition,...,additional_education_facilities,culture_objects_facilities,shopping_centers_facilities,office_num,green_part,prom_part,cafe_count,church_facilities,mosque,leisure_facilities
0,30392.0,60.0,36.000000,6.0,9.000000,1.000000,1974.000000,3.000000,5.000000,3.000000,...,3.0,0.0,16.0,1.0,2.86,0.00,0.0,0.0,0.0,0.0
1,36540.0,44.0,29.000000,9.0,13.865723,1.555441,1986.759037,1.663412,7.295711,2.464101,...,3.0,0.0,11.0,5.0,9.26,2.71,1.0,0.0,0.0,0.0
2,94669.0,62.0,60.000000,4.0,11.018363,1.635896,1985.160837,2.342162,6.786302,2.164586,...,2.0,0.0,4.0,2.0,13.80,0.00,4.0,0.0,0.0,0.0
3,17531.0,33.0,18.000000,4.0,9.000000,1.000000,1970.000000,1.000000,6.000000,2.000000,...,4.0,0.0,2.0,0.0,7.46,0.00,1.0,0.0,0.0,0.0
4,32159.0,37.0,14.000000,1.0,17.000000,1.000000,1991.000000,1.000000,1.000000,2.000000,...,4.0,0.0,1.0,0.0,0.00,0.00,3.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28995,85589.0,58.0,34.810948,2.0,9.763230,1.903057,1987.783226,1.918817,5.452792,1.852972,...,3.0,0.0,1.0,0.0,9.26,7.02,0.0,0.0,0.0,0.0
28996,11345.0,38.0,31.951249,9.0,17.000000,1.000000,2006.007224,1.000000,1.000000,1.365329,...,16.0,10.0,23.0,141.0,12.24,0.00,71.0,15.0,0.0,0.0
28997,55519.0,55.0,30.000000,9.0,16.000000,1.000000,2002.000000,2.000000,8.000000,3.000000,...,4.0,0.0,4.0,4.0,12.32,0.00,0.0,0.0,0.0,0.0
28998,17196.0,37.0,19.000000,3.0,16.000000,5.000000,1979.000000,1.000000,9.000000,3.000000,...,3.0,0.0,11.0,6.0,39.80,7.01,3.0,0.0,0.0,0.0


In [121]:
model = CatBoostRegressor(random_state=42, loss_function='RMSE', learning_rate=0.1)
n_splits = 5

# Создаем объект KFold для кросс-валидации
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Считаем кросс-валидацию с метрикой RMSE
rmse_scores = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kf))

# Выводим результаты
print("RMSE scores:", rmse_scores)
print("Среднее RMSE:", rmse_scores.mean())

0:	learn: 4826648.3604902	total: 18.3ms	remaining: 18.3s
1:	learn: 4613894.6476806	total: 26.4ms	remaining: 13.2s
2:	learn: 4434335.1491298	total: 33.3ms	remaining: 11.1s
3:	learn: 4259879.1289116	total: 40.3ms	remaining: 10s
4:	learn: 4118445.0972168	total: 47.7ms	remaining: 9.48s
5:	learn: 3985412.2641002	total: 55ms	remaining: 9.11s
6:	learn: 3864861.8696190	total: 62ms	remaining: 8.8s
7:	learn: 3759429.5832189	total: 68.3ms	remaining: 8.47s
8:	learn: 3667778.8508055	total: 75ms	remaining: 8.25s
9:	learn: 3581618.0978143	total: 81.7ms	remaining: 8.09s
10:	learn: 3513494.9754484	total: 88.7ms	remaining: 7.97s
11:	learn: 3444374.7853727	total: 95.4ms	remaining: 7.86s
12:	learn: 3384382.8116221	total: 102ms	remaining: 7.76s
13:	learn: 3331662.4714345	total: 108ms	remaining: 7.64s
14:	learn: 3284052.7977192	total: 115ms	remaining: 7.55s
15:	learn: 3241852.1932972	total: 121ms	remaining: 7.44s
16:	learn: 3203983.5405625	total: 127ms	remaining: 7.36s
17:	learn: 3168118.1065186	total: 133m